In [232]:
import os
import hashlib
import marshal
from pyeoskit import eosapi, wallet, db
from pyeoskit import config

db.reset()
config.main_token = 'UUOS'

if os.path.exists('test.wallet'):
    os.remove('test.wallet')
psw = wallet.create('test')

wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')
wallet.import_key('test', '5KFQ5opDDCADMZwwwsKF89przC9pZc9Dc4Ed2GYbGo8S32Yy2Gw')

def publish_contract(account_name, code, abi):
    m = hashlib.sha256()
    code = compile(code, "contract", 'exec')
    code = marshal.dumps(code)
    m.update(code)
    code_hash = m.hexdigest()
    r = eosapi.get_code(account_name)
    if code_hash != r['code_hash']:
        eosapi.set_contract(account_name, code, abi, 1)
    return True

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])

def run_test_code(code, account_name='test11111111'):
    publish_contract(account_name, code, abi)
    try:
        r = eosapi.push_action(account_name, 'sayhello', b'hello,world', {account_name:'active'})
        print(r['processed']['action_traces'][0]['console'])
    except Exception as e:
        print(e)
print('done!')

done!


In [234]:
eosapi.get_public_key('5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

'EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV'

In [235]:
eosapi.get_public_key('5KFQ5opDDCADMZwwwsKF89przC9pZc9Dc4Ed2GYbGo8S32Yy2Gw')

'EOS7u2gnQzoybz6YZvoi5RY6rMqkbAycDBFEFGhsvHEDmB7tffc2h'

###  send_inline(account, action_name, actor, permission, data) -> None
发送一个内联action

In [218]:
name = 'test11111111'
code = '''
import db
def apply(receiver, code, action):
    print(n2s(receiver), n2s(code), n2s(action))
    if action == N('sayhello'):
        data = read_action_data()
        print(data)
    elif action == N('sendinline'):
        receiver = N('test11111112')
        send_inline(receiver, N('sayhello'), N('test11111112'), N('active'), b'hellooooo,world')
'''
abi = ''
publish_contract(name, code, abi)

True

In [212]:
account_name = 'test11111112'
code = '''
import db
def apply(receiver, code, action):
    if action == N('sayhello'):
        data = read_action_data()
        print(data)
'''
abi = ''
publish_contract(account_name, code, abi)

True

In [231]:
name = 'test11111111'
r = eosapi.push_action(name, 'sendinline', b'hello,world', {name:'active', 'test11111112':'active'})
print(r['processed']['action_traces'][0]['console'])
print(r['processed']['action_traces'][0]['inline_traces'][0]['console'])

HttpAPIError: API returned status code: 500 {"code":500,"message":"Internal Service Error","error":{"code":3090003,"name":"unsatisfied_authorization","what":"Provided keys, permissions, and delays do not satisfy declared authorizations","details":[{"message":"transaction declares authority '${auth}', but does not have signatures for it under a provided delay of 0 ms, provided permissions ${provided_permissions}, provided keys ${provided_keys}, and a delay max limit of 3888000000 ms","file":"authorization_manager.cpp","line_number":520,"method":"check_authorization"}]}}

In [216]:
eosapi.get_account('test11111112')

{
    "account_name": "test11111112",
    "head_block_num": 1366810,
    "head_block_time": "2019-06-10T14:15:22.500",
    "privileged": false,
    "last_code_update": "2019-06-10T12:57:46.500",
    "created": "2019-06-03T09:14:25.000",
    "core_liquid_balance": "7.3000 UUOS",
    "ram_quota": 65206,
    "net_weight": 10000,
    "cpu_weight": 10000,
    "net_limit": {
        "used": 20874,
        "available": "25884847521983",
        "max": "25884847542857"
    },
    "cpu_limit": {
        "used": 20615,
        "available": "4937142836527",
        "max": "4937142857142"
    },
    "ram_usage": 4237,
    "permissions": [
        {
            "perm_name": "active",
            "parent": "owner",
            "required_auth": {
                "threshold": 1,
                "keys": [
                    {
                        "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                        "weight": 1
                    }
                ],
             

In [225]:
eosapi.get_account('test11111112')

{
    "account_name": "test11111112",
    "head_block_num": 1367633,
    "head_block_time": "2019-06-10T14:22:14.000",
    "privileged": false,
    "last_code_update": "2019-06-10T12:57:46.500",
    "created": "2019-06-03T09:14:25.000",
    "core_liquid_balance": "7.3000 UUOS",
    "ram_quota": 65206,
    "net_weight": 10000,
    "cpu_weight": 10000,
    "net_limit": {
        "used": 21217,
        "available": "25884847521640",
        "max": "25884847542857"
    },
    "cpu_limit": {
        "used": 21666,
        "available": "4937142835476",
        "max": "4937142857142"
    },
    "ram_usage": 4237,
    "permissions": [
        {
            "perm_name": "active",
            "parent": "owner",
            "required_auth": {
                "threshold": 1,
                "keys": [
                    {
                        "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                        "weight": 1
                    }
                ],
             

In [221]:
account_name = 'test11111112'
a = {
    "account": account_name,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":'test11111111',"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = eosapi.push_action('eosio', 'updateauth', a, {account_name:'owner'})
print('done!')

done!


In [233]:
account_name = 'test11111112'
a = {
    "account": account_name,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                "weight": 1
            },
        ],
        "accounts": [],
        "waits": []
    }
}
r = eosapi.push_action('eosio', 'updateauth', a, {account_name:'owner'})
print('done!')


done!


In [229]:
eosapi.push_action('eosio', 'setpriv', {'account':'test11111111', 'is_priv':0}, {'eosio':'active'})

{'transaction_id': 'fc6a551f524d16806997ab1dce1a0b85b6b0819b57deb47b87423aba4a3c5614',
 'processed': {'id': 'fc6a551f524d16806997ab1dce1a0b85b6b0819b57deb47b87423aba4a3c5614',
  'block_num': 1367754,
  'block_time': '2019-06-10T14:23:14.500',
  'producer_block_id': None,
  'receipt': {'status': 'executed',
   'cpu_usage_us': 401,
   'net_usage_words': 13},
  'elapsed': 401,
  'net_usage': 104,
  'scheduled': False,
  'action_traces': [{'receipt': {'receiver': 'eosio',
     'act_digest': 'b0f6b413527631cc952fd802e3d5cdd6c358c750872bf42040fcbcf5f41ef910',
     'global_sequence': 1369825,
     'recv_sequence': 1368702,
     'auth_sequence': [['eosio', 1367932]],
     'code_sequence': 1,
     'abi_sequence': 1},
    'act': {'account': 'eosio',
     'name': 'setpriv',
     'authorization': [{'actor': 'eosio', 'permission': 'active'}],
     'data': {'account': 'test11111111', 'is_priv': 0},
     'hex_data': '104208218490b1ca00'},
    'context_free': False,
    'elapsed': 227,
    'console': 

In [230]:
eosapi.get_account('test11111111')

{
    "account_name": "test11111111",
    "head_block_num": 1367763,
    "head_block_time": "2019-06-10T14:23:19.000",
    "privileged": false,
    "last_code_update": "2019-06-10T14:19:22.000",
    "created": "2019-06-03T09:13:59.000",
    "core_liquid_balance": "972367.5412 UUOS",
    "ram_quota": 65206,
    "net_weight": 10000,
    "cpu_weight": 10000,
    "net_limit": {
        "used": 47540,
        "available": "25884847495317",
        "max": "25884847542857"
    },
    "cpu_limit": {
        "used": 94793,
        "available": "4937142762349",
        "max": "4937142857142"
    },
    "ram_usage": 52365,
    "permissions": [
        {
            "perm_name": "active",
            "parent": "owner",
            "required_auth": {
                "threshold": 1,
                "keys": [
                    {
                        "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                        "weight": 1
                    }
                ],
       